In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep
import pandas as pd
import random
import os

In [2]:
version = "1.0.0"
datadir = "data/"
archivedir = "data/Archive"

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [4]:
def gedjournalarticles(filename):
    # open old file
    olddf = pd.DataFrame({'url': [] })
    try:
        olddf = pd.read_pickle(os.path.join(archivedir, filename))
        print(olddf.tail())
    except:
        pass

    driver.get("https://scholarship.claremont.edu/jhm/all_issues.html")
    
    # get all of the issue
    def issues():
        issues = []
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # get all of the issue div
        for h3 in soup.find_all('h3', {'class': 'issue'}):
            for a in h3.find_all('a'):                
                issues.append(a['href'])        
        
        return  issues    
    allissue = issues()
    allissue = list(set(allissue))
    print(allissue)
    
    def articles():
        articles = []
        nexpage = False
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # We collect the texts under the Articles         
        # get the 'Artilces' h2
        for h2 in soup.find_all('h2'):
            if h2.string != "Articles":
                continue
            for sib in h2.next_siblings:
                if sib.name == 'div':
                    for a in sib.find_all("a"):
                        if not re.search("cgi", a['href']):
                            articles.append(a['href'])
                elif sib.name == 'h2':
                    break
        return  articles
    
    # iterate over isses and get article links
    articellinks = []
    for issue in allissue:
        driver.get(issue)
        sleep(2)
        print(driver.current_url, end='\r')         
        thispagearticles = articles()
        articellinks = articellinks + thispagearticles        
    articellinks = list(set(articellinks))
    print(articellinks)
    
    # filter out old articles
    oldarticles = olddf['url']
    articellinks = list( set(articellinks).difference(set(oldarticles) ))
    print(len(articellinks), "new article found!")
    
    # iterate over article 
    urls = [ None for _ in range(len(articellinks))]
    titles = [ None for _ in range(len(articellinks))]
    abstracts = [ None for _ in range(len(articellinks))]
    writers = [ None for _ in range(len(articellinks))]
    dates = [ None for _ in range(len(articellinks))]
    dois = [  None for _ in range(len(articellinks)) ]
    keywords = [  None for _ in range(len(articellinks)) ]
    for idx in range(len(articellinks)):
        # print percentiage of the process
        print( str(np.round(100*idx/len(articellinks),2))+"%" , end='\r')
        url =  articellinks[idx]
        # load page
        driver.get(articellinks[idx])
        sleep(3)        
        urls[idx] = driver.current_url
        # get the issue article
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # process metadata
        keyword = []
        writter = []
        abstract = ""
        for m in soup.find_all("meta"):
            if m.has_attr("name"):
                if m['name'] == "keywords":
                    keyword = m['content'].split(",")
                if m['name'] == "bepress_citation_online_date":
                    dates[idx] = datetime.datetime.strptime(m['content'], "%Y/%m/%d").strftime('%Y-%m-%d')
                if m['name'] == 'author':
                    writter.append(m['content'])
                if m['name'] == 'bepress_citation_doi':
                    dois[idx] = m['content']
                if m['name'] == 'description':
                    abstracts[idx] = abstract = m['content']
            if m.has_attr("property"):                    
                if m['property'] == 'og:title':
                    titles[idx] = m['content']
        if len(writter) > 0:
            writers[idx] =  "#".join(writter)
        if len(keyword) > 0:
            keywords[idx] =  "#".join(keyword)
    
    # make df
    df = pd.DataFrame({'url': urls, 
                       'journal_title': "Journal of Humanistic Mathematics", 
                       'journal_eissn': "2159-8118",
                       'journal_pissn': '',
                       'category': "Mathematics",                                              
                      })
    # extend the df
    df['title'] = titles
    df['doi'] = dois
    df['abstract'] = abstracts
    df['writer'] = writers
    df['publishdate'] = dates
    df['keyword'] = keywords
    
    # drop where is no abstract
    df = df[ df['abstract'].notna()]
    # drop where title is Editorial
    df = df[df['title'] != 'Editorial']    
    
    # check there was a not Arhived but previously loaded file
    adf = None
    try:
        adf = pd.read_pickle(os.path.join(datadir, filename))
    except:
        pass
    if adf is not None:
        df = pd.concat([df, adf])
        df.drop_duplicates(inplace=True)
    
    return df, olddf

In [ ]:
filename = 'journal_Journal_of_Humanistic_Mathematics_'+version+'.pandas'
    
# search for articles
df, olddf = gedjournalarticles( filename )

# save data
df.to_pickle(os.path.join(datadir, filename))

print(df.head())


['https://scholarship.claremont.edu/jhm/vol7/iss2/', 'https://scholarship.claremont.edu/jhm/vol9/iss2/', 'https://scholarship.claremont.edu/jhm/vol10/iss1/', 'https://scholarship.claremont.edu/jhm/vol3/iss2/', 'https://scholarship.claremont.edu/jhm/vol5/iss1/', 'https://scholarship.claremont.edu/jhm/vol11/iss1/', 'https://scholarship.claremont.edu/jhm/vol6/iss2/', 'https://scholarship.claremont.edu/jhm/vol4/iss2/', 'https://scholarship.claremont.edu/jhm/vol2/iss2/', 'https://scholarship.claremont.edu/jhm/vol8/iss2/', 'https://scholarship.claremont.edu/jhm/vol6/iss1/', 'https://scholarship.claremont.edu/jhm/vol8/iss1/', 'https://scholarship.claremont.edu/jhm/vol1/iss1/', 'https://scholarship.claremont.edu/jhm/vol3/iss1/', 'https://scholarship.claremont.edu/jhm/vol9/iss1/', 'https://scholarship.claremont.edu/jhm/vol4/iss1/', 'https://scholarship.claremont.edu/jhm/vol5/iss2/', 'https://scholarship.claremont.edu/jhm/vol1/iss2/', 'https://scholarship.claremont.edu/jhm/vol2/iss1/', 'https://

In [ ]:
print(df.iloc[0]['abstract'])

In [ ]:
print(df.iloc[0])

In [ ]:
driver.close()

In [ ]:
print(df['keyword'].values.tolist())